In [1]:
import numpy as np
import tkinter as tk
from tkinter import messagebox
from tkinter import scrolledtext
waitlist=[]
lifadiansave=0
guest_amount=0
t_counter=0
t_jiange=0
t_counter=0
t_arrival_jiange=0
t_shengyu=0
t_wait=0
timesum=0
status=0
t_max=0
wait_signal=0
renshu=0
lifat=0

In [2]:
#客人生成相关模块
def lifatime():#理发时间生成函数
    time= int(29*np.random.random())+1
    return time
def guest(guest_amount):#客人定义函数
    guesttotal=[]
    guest_name=guest_amount
    guesttotal.append(guest_name)
    guest_lifatime=lifatime()
    guesttotal.append(guest_lifatime)
    guest_arrivaltime=t_start+int(20*np.random.random())+1#定义客人到来的间隔是（0-10min）
    guesttotal.append(guest_arrivaltime)
    #guest_leavetime=guest_arrivaltime+guest_lifatime
    return guesttotal
def guest_generator():#客人生成器
    global guest_amount
    guest_amount=guest_amount+1
    guest_temp=guest(guest_amount)
    #print("顾客",guest_temp[0],"已生成")
    #print("预期到达时间为：",guest_temp[2])
    #print("预期的理发时间为:",guest_temp[1])
    global t_start
    t_start=guest_temp[2]
    return guest_temp

In [3]:
#总控信号系统
def centercontrol():#理发店总控编辑，管理总的系统,负责指挥信号的发送
    global center_avasignal
    if sum(Barberstatus)<K:#理发店列表初始均为0,0表示有空,1表示占用，只要总和<K，说明有空
        center_avasignal=0
        scr1.insert(tk.INSERT,"目前理发店内有空位\n")
    if sum(Barberstatus)==K:
        center_avasignal=1
        scr1.insert(tk.INSERT,"目前理发店内没有空位\n")
def waitcontrol():#等待区控制系统，若有空位发送信号0，无空位发送信号1
    global wait_signal
    global have_signal#是否有人在等待
    if len(waitlist)<L:
        wait_signal=0
    if len(waitlist)==L:
        wait_signal=1
    if len(waitlist)!=0:
        have_signal=1
    else:
        heve_signal=0

In [4]:
def waitbench(a):
    if center_avasignal==0:
        if len(waitlist)!=0:
            temp = waitlist[0]
            waitlist.pop(0)
            scr1.insert(tk.INSERT,"客人{}进入理发店\n".format(temp[0]))
            return temp
        else:
            return a
    if wait_signal==0:
        waitlist.append(a)
        if center_avasignal==1:
            scr1.insert(tk.INSERT,"理发店全员运转，客人开始等待\n".format(a[0]))
            global counter
            counter=counter+1
    if wait_signal==1:
        scr1.insert(tk.INSERT,"等待区域已经满员,客人{}未理发离开\n".format(a[0]))

In [5]:
class Barber:#理发店内理发师作为单独的节点进行编辑
    def __init__(self):
        self.guestname=None
        self.lifatime=None
        self.starttime=None
    def feedback(self):
        if self.guestname==None:
            return 0
        else:
            return 1
    def t_solver(self):
        if self.lifatime==None:
            return 0
        else:
            t_shengyu=self.lifatime
            return t_shengyu
    def reset(self):
        self.guestname=None
        self.lifatime=None
        self.starttime=None
    def end(self):
        endtime=self.starttime+self.lifatime
        scr1.insert(tk.INSERT,"顾客{}结束理发\n".format(self.guestname))
        scr1.insert(tk.INSERT,"当前时间为{}\n".format(endtime))
        global renshu
        global lifat
        renshu=renshu+1
        lifat=lifat+self.lifatime
        Barber.reset(self)
        Barber.feedback(self)
        return endtime 

In [6]:
# 理发店总控系统,总控系统初始化
t_counter=0
Barberstatus=[]#用于检测是否有空余节点
shoptime_solver=[]#用于时间的记录，并进行相关的计算
Barberlist=[]
def new():
    global Barberlist
    global shoptime_solver
    global Barberstatus
    for i in range(0,K):
        x=Barber()
        Barberlist.append(x)
        y=x.t_solver()
        shoptime_solver.append(y)
        z=x.feedback()
        Barberstatus.append(z)
def fresh(i):#实时更新状态
    Barberstatus[i]=Barberlist[i].feedback()
    shoptime_solver[i]=Barberlist[i].t_solver()

In [7]:
def start():
    global counter,biaozhun,chushi,have_signal,t_counter
    counter=0
    old_counter=-1
    chushi=guestsavelist[0]
    biaozhun=0
    have_signal=0
    if chushi[0]==1:
        t_counter=chushi[2]
    scr1.insert(tk.INSERT,"状态初始化，当前时间为：{}\n".format(t_counter))
    while True:
        if counter==len(guestsavelist):
            for i in range(0,len(Barberstatus)):
                if Barberstatus[i]!=0:
                    Barberlist[i].end()
            break
        guest=guestsavelist[counter]
        scr1.insert(tk.INSERT,"客人{}预期到达时间为{}\n".format(guest[0],guest[2]))
        #print("counter",counter)
        t_arrival_jiange=guest[2]-t_counter#新客人与前一位客人的到来间隔
        #print("t_arrival_jiange",t_arrival_jiange)
        t_shengyu=min(shoptime_solver)
        #print("t_shengyu",t_shengyu)
        templist=[]
        while True:
            for i in shoptime_solver:
                if i!=0:
                    templist.append(i)
            #print(templist)
            if(len(templist)!=0):
                biaozhun=min(templist)
            #print("标准",biaozhun)
            if t_arrival_jiange>=biaozhun and biaozhun!=0:
                for i in range(0,len(shoptime_solver)):
                    if Barberlist[i].feedback()==1:
                        shoptime_solver[i]=shoptime_solver[i]-biaozhun
                        if shoptime_solver[i]==0:
                            t_counter=Barberlist[i].end()
                            fresh(i)
                #print("状态更新中，当前时间为：",t_counter)
                t_arrival_jiange=t_arrival_jiange-biaozhun
                templist=[]
            elif biaozhun==0:
                break
            else:
                centercontrol()
                waitcontrol()
                if have_signal==0:
                    t_counter=guest[2]
                #print(have_signal)
                for i in range(0,len(shoptime_solver)):
                    if Barberlist[i].feedback()==1:
                        shoptime_solver[i]=shoptime_solver[i]-t_arrival_jiange
                break
        centercontrol()    
        waitcontrol()
        oldjudge=wait_signal
        a=waitbench(guest)
    
        for i in range(0,len(Barberlist)):
            if Barberlist[i].feedback()==0:
                Barberlist[i].guestname=a[0]
                Barberlist[i].lifatime=a[1]
                Barberlist[i].starttime=t_counter
                scr1.insert(tk.INSERT,"{}号客人开始理发\n".format(a[0]))
                scr1.insert(tk.INSERT,"当前时间为:{}\n".format(t_counter))
                scr1.insert(tk.INSERT,"理发师{}号为您服务。\n".format(i))
                scr1.insert(tk.INSERT,"预期理发时间为：{}\n".format(a[1]))
                temp=1
                #print(counter)
                fresh(i)
                break
        if wait_signal==1:
            counter=counter+1
            continue
        centercontrol()
        waitcontrol()
        if oldjudge==1 and wait_signal==0:
            a=waitbench(guest)
            for i in range(0,len(Barberlist)):
                if Barberlist[i].feedback()==0:
                    Barberlist[i].guestname=a[0]
                    Barberlist[i].lifatime=a[1]
                    Barberlist[i].starttime=t_counter
                    scr1.insert(tk.INSERT,"客人开始理发\n".format(a[0]))
                    scr1.insert(tk.INSERT,"当前时间为:{}\n".format(t_counter))
                    scr1.insert(tk.INSERT,"理发师{}号为您服务。\n".format(i))
                    scr1.insert(tk.INSERT,"预期理发时间为：{}\n".format(a[1]))
                    temp=1
                    #print(counter)
                    fresh(i)
                    break
        else:
            pass
    
        if temp==1:
            counter=counter+1
            #print(counter)
        elif t_counter>guest[2]:
            counter=counter+1
            #print(counter)
        temp=0
        scr1.insert(tk.INSERT,"  \n")

In [8]:
def new_start():
    global guestsavelist
    global waitlist
    global lifadiansave
    global guest_amount
    global t_counter
    global t_jiange
    global t_counter
    global t_arrival_jiange
    global t_shengyu
    global t_wait
    global timesum
    global status
    global t_max
    global wait_signal
    global t_counter
    global Barberstatus
    global shoptime_solver
    global Barberlist
    global renshu
    global lifat
    renshu=0
    lifat=0
    guestsavelist=[]
    waitlist=[]
    lifadiansave=0
    guest_amount=0
    t_counter=0
    t_jiange=0
    t_counter=0
    t_arrival_jiange=0
    t_shengyu=0
    t_wait=0
    timesum=0
    status=0
    t_max=0
    wait_signal=0
    t_counter = 0
    Barberstatus=[]#用于检测是否有空余节点
    shoptime_solver=[]#用于时间的记录，并进行相关的计算
    Barberlist=[]
    scr.delete(0.0,tk.END)
    scr1.delete(0.0,tk.END)
    messagebox.showwarning("通知","可以重新输入了")

In [9]:

root_window=tk.Tk()
root_window.title('大树十字坡理发店模拟器')
root_window.geometry('1500x1500')
scr = scrolledtext.ScrolledText(root_window, width=40, height=20,font=("宋体",18))  #滚动文本框（宽，高（这里的高应该是以行数为单位），字体样式）
scr.grid(row=16,column=0) #滚动文本框在页面的位置
scr1 = scrolledtext.ScrolledText(root_window, width=40, height=20,font=("宋体",18))  #滚动文本框（宽，高（这里的高应该是以行数为单位），字体样式）
scr1.grid(row=16,column=3) #滚动文本框在页面的位置
root_window["background"] = "white"
text=tk.Label(root_window,text="大树十字坡理发店模拟器",bg="white",fg="black",font=('Times', 20))
text.grid(row=0,column=2)
text1=tk.Label(root_window,text="客人情况显示器",bg="white",fg="black",font=('Times', 15))
text1.grid(row=15,column=0)
text2=tk.Label(root_window,text="运营情况显示器",bg="white",fg="black",font=('Times', 15))
text2.grid(row=15,column=3)
labe1 = tk.Label(root_window,text="请输入开门时间（整数）：",bg="white",fg="black")
labe2 = tk.Label(root_window,text="请输入关门时间（整数）：",bg="white",fg="black")
labe3 = tk.Label(root_window,text="请输入理发师人数（整数）：",bg="white",fg="black")
labe4 = tk.Label(root_window,text="请输入最大等待人数（整数）：",bg="white",fg="black")
labe1.grid(row=1)
labe2.grid(row=2)
labe3.grid(row=3)
labe4.grid(row=4)
type1=tk.IntVar()
type2=tk.IntVar()
type3=tk.IntVar()
type4=tk.IntVar()
entry1 = tk.Entry(root_window,textvariable=type1)
entry2 = tk.Entry(root_window,textvariable=type2)
entry3 = tk.Entry(root_window,textvariable=type3)
entry4 = tk.Entry(root_window,textvariable=type4)
entry1.grid(row=1, column=1)
entry2.grid(row=2, column=1)
entry3.grid(row=3, column=1)
entry4.grid(row=4, column=1)
def output1():
    output1=tk.Label(root_window,text="今天服务的理发人数为{}".format(renshu),bg="white",fg="black",font=('Times', 15))
    output1.grid(row=40,column=2)
    output2=tk.Label(root_window,text="今天服务的理发平局时间为为{}".format(lifat/renshu),bg="white",fg="black",font=('Times', 15))
    output2.grid(row=41,column=2)
def input1():
    global t_start
    global t_end
    global K
    global L
    try:
        t_start=int(entry1.get())
        t_end=int(entry2.get())
        K=int(entry3.get())
        L=int(entry4.get())
    except:
        messagebox.showwarning('警告','请输入大于0整数数字')
    if t_start<0 or t_end<t_start or K<0 or L<0:
        messagebox.showwarning('警告','请检查逻辑，输入大于0的整数数字')
    new()
def guiguest():
    global guestsavelist
    guestsavelist=[]
    scr.insert(tk.INSERT,"    客人开始生成     \n")
    while True:
        a=guest_generator()
        scr.insert(tk.INSERT,"顾客{}已经生成 \n".format(a[0]))
        scr.insert(tk.INSERT,"顾客预计理发时间为{}\n".format(a[1]))
        scr.insert(tk.INSERT,"顾客预计到达时间为{}\n".format(a[2]))
        scr.insert(tk.INSERT,"\n")
        if a[2]>t_end:#到来时间超过关门时间，停止客人的生成
            scr.insert(tk.INSERT,"客人{}到来时已经是关门时间,不能理发\n".format(a[0]))
            break
        guestsavelist.append(a)
tk.Button(root_window,text='输入确定',width=10,command=input1).grid(row=12,column=0,sticky=tk.W,padx=10,pady=5)
tk.Button(root_window,text='客人生成',width=10,command=guiguest).grid(row=12,column=1,sticky=tk.W,padx=10,pady=5)
tk.Button(root_window,text='开始运营',width=10,command=start).grid(row=12,column=2,sticky=tk.W,padx=10,pady=5)
tk.Button(root_window,text='重新启动',width=10,command=new_start).grid(row=12,column=3,sticky=tk.W,padx=10,pady=5)
tk.Button(root_window,text='今日小结',width=10,command=output1).grid(row=39,column=2,sticky=tk.W,padx=10,pady=5)
root_window.mainloop()#开启主循环，让窗口处于显示状态